<a href="https://colab.research.google.com/github/rockingboi/prodigy_internship/blob/main/Hand_Gesture_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'leapgestrecog:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F39466%2F61155%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240320%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240320T163921Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D055737ea2c165642e8d0e88b81fec7fe7df97ded6906ce3ed3475d82453ad36d07c5f5bc78f923991ac0634ba0c0e2cbdbad5521c51090bc30410bddc1cae65c515a4a3883e756fd379425b103dcc72ef871c50f0c81ec66875e4f4fc7364d2b37dcf4dbfd6928cae0faab82de9cc5cd7aabad7d44232ccff772fb3a03aaf441b250a4959d5d6a2fc28a38735a5d42a7aa4991954ba805bde187f84fe726f10916ae9a54cd395b67f7602046c4fdcb4f9c45d7fedbf0ff3f3d128021c8a73b713cd5fe755017dda2e09a6efc151ed17bf358ba0fa31064b0637a9ace13ff79b7e98c0d766a313841e2db356168ad323a7bf500bb659c7893edac0521751f996c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


### 📍📍 About the Data

# 📌Importing Libaraies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.math import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
tf.random.set_seed(3)
import glob
import skimage as oi
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, LeakyReLU


# 📌 Loading The Data

In [ ]:
folders_names = []
##'/kaggle/input/leapgestrecog/leapGestRecog/0'
for i in range(10):
    folder = r'C:\Users\user\Desktop\archive (3)\leapGestRecog\0{}'.format(i)
    folders_names.append(folder)

files_names = ['01_palm', '02_l', '03_fist', '04_fist_moved', '05_thumb']


In [ ]:
folders_names

### 📍📍Display 3 images for each file from eav=ch class

In [ ]:
for folder in folders_names:
        Class_num=folder[-1]
        for file in files_names:
            path = os.path.join(folder, file)
            x=0
            fig, axes = plt.subplots(1,3, figsize=(25, 4))
            for img in os.listdir(path):


                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                axes[x].imshow(img_array, cmap='gray')
                x += 1
                if x == 3:
                    break

            plt.suptitle(f"Class{Class_num} , {file}", fontsize=26)
            plt.show()

### 📍📍loading

In [ ]:
training_data = []

def create_training_data():
    for folder in folders_names:
        Class_num=folder[-1]
        print('Class ' ,Class_num)
        for file in files_names:
            path = os.path.join(folder, file)

            print('Class ' ,Class_num,file)
#             c=0

            for img in tqdm(os.listdir(path)):
#                 if(c==100):
#                     break
#                 c+=1
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)


                training_data.append([img_array,int(Class_num)])


create_training_data()

# 📌Sizes of images

### 📍📍 printing the shape of the first image from each class

In [ ]:
for folder in folders_names:
    class_num = folder[-1]
    print('Class', class_num)
    for file in files_names:
        path = os.path.join(folder, file)

        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            print("Image shape:", img_array.shape,"\n")
            break  # To print only the first image shape for each file
        break  # To print only the first file shape for each folder


### 📍📍 Checking if all images have the same size or not

In [ ]:
def check_image_sizes():
    first_img_shape = None
    for folder in folders_names:
        for file in files_names:
            path = os.path.join(folder, file)
            for img in os.listdir(path):
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                if first_img_shape is None:
                    first_img_shape = img_array.shape
                elif img_array.shape != first_img_shape:
                    print("Image sizes are not consistent.")
                    return False
    print("All images have the same size:", first_img_shape)

check_image_sizes()


# 📌Shuffling

### 📍📍getting the classes of the first 5 images

In [ ]:
for i in range(5):
    print("Class number for image", i+1, ":", training_data[i][1])

### 📍📍getting the classes of the lastt 5 images

In [ ]:
for i in range(-1, -6, -1):
    print("Class number for image", len(training_data) + i + 1, ":", training_data[i][1])

### 📍📍Suffling and display the first 10 classes after suffling

In [ ]:
random.shuffle(training_data)

for i in range(15):
    print(f"Sample {i+1}:")
    print("Class number:", training_data[i][1],"\n")

# 📌Preprocessing

### 📍📍Feature Selection

In [ ]:
X=[]
y=[]

for feature,label in training_data:
    X.append(feature)
    y.append(label)


In [ ]:
print(type(X))
print(type(y))


### 📍📍Convert X ,y from list to numpy array

In [ ]:
X=np.array(X)
y=np.array(y)
print(X.shape)
print(y.shape)

### 📍📍Spliting the Data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

### 📍📍Getting classes names

In [ ]:
print(np.unique(y_train))

print(np.unique(y_test))

### 📍📍Scalling the Data

In [ ]:
X_train = X_train/255
X_test = X_test/255

### 📍📍print first image after scalling

In [ ]:
print(X_train[0])

# 📌Modeling Time

### 📍📍Building Multi Neural Network

In [ ]:

model = Sequential([
    Flatten(input_shape=(240, 640)),
#     Dense(1024),
#     LeakyReLU(alpha=0.1),
#     Dense(512),
#     LeakyReLU(alpha=0.1),
#     Dense(256),
#     LeakyReLU(alpha=0.1),
#     Dense(128),
#     LeakyReLU(alpha=0.1),
    Dense(64),
    LeakyReLU(alpha=0.1),
    Dense(32),
    LeakyReLU(alpha=0.1),
    Dense(16),
    LeakyReLU(alpha=0.1),
    Dense(10, activation='softmax')
])

### 📍📍Compiling the model

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### 📍📍Training the Neural Network

In [ ]:
history = model.fit(X_train, y_train, epochs=3, validation_split=0.1, batch_size=32, verbose=2)


### 📍📍 Model Summary

In [ ]:
model.summary()

# 📌Evaluation time

### 📍📍Training Loss

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train)
print(f"Training Loss: {loss:.4f}")
print(f"Training Accuracy: {accuracy*100:.2f}%")


### 📍📍Testing Loss

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Testing Loss: {loss:.4f}")
print(f"Testing Loss : {accuracy*100:.2f} %")

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### 📍📍Prediction

In [ ]:
y_pred=model.predict(X_test)
print(y_pred[0])

In [ ]:
y_pred = [np.argmax(i) for i in y_pred]
print(y_pred[1])

### 📍📍y_test Vs y_pred

In [ ]:
comparison_df = pd.DataFrame({ 'Actual': y_test,'Predicted': y_pred})

print(comparison_df[:20])

### 📍📍 Confusion Matrix

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred)
print(conf_mat)

In [ ]:
plt.figure(figsize=(15,7))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='bone')
plt.ylabel('True Labels')
plt.xlabel('Predicted Labels')

### 📍📍 Classification Report

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))